## **Load libraries and data**

In [1]:
!pip install --upgrade pythainlp
!pip install pyLDAvis

In [2]:
import pandas as pd
import pythainlp
import gensim
import pyLDAvis.gensim_models
pyLDAvis.enable_notebook()
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

D:\py\envs\7105\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [3]:
review_file_path = 'Wongnai Reviews - Small.csv'
review_file_path_name = review_file_path[0:review_file_path.rfind('.')]

In [4]:
df = pd.read_csv(review_file_path)

In [5]:
df.tail()

,Review ID,Review
295,296,ค่ำนี้คุณเพื่อนอยากส้มตำ หมูเฮาเลยพากันลงมากิน...
296,297,ร้านสะอาดดี ตกแต่งสวยงาม มีที่จอดรถ ราคาเมนูต่...
297,298,เช้าๆ รีบๆ วิ่งมาเข่าห้องเรียนแทบไม่ทันแต่ต้อง...
298,299,ร้านนี้เป็นร้านกาแฟเล็กๆ ข้างๆ ร้านๆ Happy Man...
299,300,ทรูคอฟฟี่สาขาซีคอนอยู่ในศูนย์บริการของทรู ชั้น...


## **Tokenize and Normalize Thai Words**

In [6]:
stopwords = list(pythainlp.corpus.thai_stopwords())
removed_words = [ 'ๆ', 'ร้าน', 'ผม', 'เรา', 'ตัวเรา', 'พวกเรา', 'นนน', 'กกก', '  ', ' ', '"', '\n', '\r\n', '//', '/', '..', '.', '(', ')', '!!', '!', '--', '-' '++', '+', '??', '?', '**', '*', '>', '<', ':', '@']
screening_words = stopwords + removed_words

print(screening_words)

['เห็นว่า', 'หรือไร', 'เนื่องจาก', 'มองว่า', 'เช่นดังที่', 'อันได้แก่', 'ทั่ว', 'ยืนนาน', 'ตาม', 'อันไหน', 'จัง', 'ยัง', 'จนกว่า', 'ด้วยเพราะ', 'ถึงเมื่อไร', 'เหล่านี้', 'ดังกับว่า', 'ที่ซึ่ง', 'ยิ่งขึ้นไป', 'ที่ๆ', 'ส่วนใด', 'ส่วนที่', 'ละ', 'บ่อยๆ', 'ใช้', 'เชื่อ', 'วันนี้', 'กระนั้น', 'ก็แค่', 'ช่วงๆ', 'ครั้งละ', 'นอกเหนือจาก', 'ไม่ค่อยเป็น', 'โตๆ', 'อย่างมาก', 'ถึงแก่', 'หรือไง', 'ที่แห่งนั้น', 'เสียก่อน', 'กว่า', 'พยายาม', 'จัดงาน', 'ข้างล่าง', 'ทั้งเป็น', 'อย่างไหน', 'เกี่ยวกัน', 'แล้ว', 'ตลอดวัน', 'หนึ่ง', 'ถ้าหาก', 'พวกเธอ', 'แห่งนั้น', 'ขณะหนึ่ง', 'ช่วงหลัง', 'ค่อยไปทาง', 'นั่นไง', 'ทุกเมื่อ', 'พอสม', 'เธอ', 'ทุกที่', 'แค่เพียง', 'สิ่งใด', 'ขวางๆ', 'บางที', 'พร้อมด้วย', 'ซะจน', 'ถึงจะ', 'ข้าฯ', 'นี้เอง', 'มั้ยล่ะ', 'ตลอดทั่วทั้ง', 'เป็นต้นไป', 'ทำๆ', 'บอกว่า', 'หน่อย', 'ข้างๆ', 'ดังเก่า', 'ถึงแม้ว่า', 'แม้นว่า', 'ตลอดปี', 'นี่ไง', 'ใน', 'ทำไร', 'ก็', 'เสียยิ่ง', 'อย่างไรก็', 'เกิน', 'กับ', 'ใหม่', 'นี้', 'คล้าย', 'ผ่าน', 'อาจเป็น', 'คง', 'ทั้งสิ้น', 'บ่อยครั้ง', 'พอจะ', 'อย่าง

In [7]:
correctwords = {
    '฿': 'บาท',
    '&': 'และ',
    'ก้อ': 'ก็',
    'เก้': 'เก๋',
    'เก๊': 'เก๋',
    'กทม.': 'กรุงเทพฯ',
    'กรุงเทพ': 'กรุงเทพฯ',
    'เค็ก': 'เค้ก',
    'จิง': 'จริง',
    'เจง': 'จริง',
    'ดี๊': 'ดี',
    'เข่ง': 'เข็ง',
    'มากก': 'มาก',
    '^^': 'ชอบ',
    'ฟินน': 'ชอบ',
    'ฟิน': 'ชอบ',
}

In [8]:

def tokenize_with_space(sentence):
  merged = ''
  words = pythainlp.word_tokenize(str(sentence), engine='newmm')
  for word in words:
    for remword in removed_words:
      if word.find(remword) >= 0:
        word = word.replace(remword, '', 30)
    if word in correctwords:
      word = correctwords[word]
    if word not in screening_words and len(word) >= 2:
      merged = merged + ',' + word
  return merged[1:]

In [9]:
df['Review_tokenized'] = df['Review'].apply(lambda x: tokenize_with_space(x))

df.to_csv(review_file_path_name + '-tokenized.csv')

In [10]:
df.tail()

,Review ID,Review,Review_tokenized
295,296,ค่ำนี้คุณเพื่อนอยากส้มตำ หมูเฮาเลยพากันลงมากิน...,"ค่ำ,เพื่อน,ส้มตำ,หมู,เฮา,ลงมา,กิน,ส้มตำ,ออฟฟิศ..."
296,297,ร้านสะอาดดี ตกแต่งสวยงาม มีที่จอดรถ ราคาเมนูต่...,"สะอาด,ดี,ตกแต่ง,สวยงาม,ที่จอดรถ,ราคา,เมนู,เทีย..."
297,298,เช้าๆ รีบๆ วิ่งมาเข่าห้องเรียนแทบไม่ทันแต่ต้อง...,"เช้า,รีบ,วิ่ง,เข่า,ห้องเรียน,แทบ,ต้องหา,ของกิน..."
298,299,ร้านนี้เป็นร้านกาแฟเล็กๆ ข้างๆ ร้านๆ Happy Man...,"กาแฟ,Happy,Mango,อาทิตย์,นัด,เพื่อน,นั่ง,คุย,ด..."
299,300,ทรูคอฟฟี่สาขาซีคอนอยู่ในศูนย์บริการของทรู ชั้น...,"ทรู,คอ,ฟี่,สาขา,ซีคอน,ศูนย์,บริการ,ทรู,ชั้น,โซ..."


## **Create Dictionary**

In [11]:
documents = df['Review_tokenized'].to_list()
texts = [[text for text in doc.split(',')] for doc in documents]
dictionary = gensim.corpora.Dictionary(texts)

In [12]:
print(dictionary.token2id.keys())

ont', 'No', 'Portico', 'Potato', 'Raspberry', 'Sakura', 'Twinings', 'Yakiniku', 'Yuzu', 'farm', 'กระปรี้กระเปร่า', 'กลมกลืน', 'กานา', 'ของดี', 'คลายอารมณ์', 'ความนุ่ม', 'ความสวยงาม', 'ค์บ', 'จับตา', 'จาง', 'ฉ่ำ', 'ชุ่มชื่น', 'ซากุระ', 'ซุ', 'ฐาน', 'ดังนี้', 'ตั๋ว', 'ตึ๊บ', 'ทับ', 'ทาร์ต', 'ทำให้เกิด', 'ทิ', 'น่าดู', 'น้ำแร่', 'ประกบ', 'ประจำเมือง', 'ประณีต', 'ประดับ', 'ปลื้มใจ', 'ปาด', 'ปุปะ', 'ผลักดัน', 'ผสมผสาน', 'ผิว', 'ผ่อน', 'ฟิล', 'ฟุต', 'มโหฬาร', 'ยุโรป', 'ระยะหลัง', 'ราสเบอร์รี่', 'รู้ใจ', 'ลิ้นจี่', 'วาไรตี้', 'วิเคราะห์', 'สตรอเบอร์รี่', 'สภาพ', 'สอด', 'สังสรรค์', 'หนักหน่วง', 'ออกปาก', 'อัศจรรย์', 'อำพัน', 'อิ่มเอม', 'ฮอง', 'เกาลัด', 'เข้าด้วยกัน', 'เชื่อม', 'เป็นมา', 'เหนื่อยล้า', 'เอร็ดอร่อย', 'แชมเปี้ยน', 'แดงเข้ม', 'แตร์', 'แทรก', 'แหลม', 'แอพพริคอท', 'โมจิ', 'โอกินาวา', 'ใส้', 'ความขลัง', 'ทดลอง', 'สังขยา', 'โอเลี้ยง', 'ไปรษณีย์', 'Japan', 'Larb', 'you', 'กลิ่นอาย', 'ข้าวปั้น', 'ชาม', 'ช่างคิด', 'ตคอ', 'ตั๊กแตน', 'บมิ', 'บางนา', 'ปุระ', 'ยำสลัด', 'รีรอ', 'วาซาบิ', 'ว้าว

In [13]:
file=open(review_file_path_name + '-dict.txt', 'w', encoding='utf8')
for item in dictionary.token2id.keys():
    file.write(str(item) + '\n')
file.close()


In [14]:
gensim_corpus = [dictionary.doc2bow(text, allow_update=True) for text in texts]
word_frequencies = [[(dictionary[id], frequence) for id, frequence in couple] for couple in gensim_corpus]
print(gensim_corpus[0])
print(word_frequencies[0])

[(0, 1), (1, 2), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 2), (26, 2), (27, 1)]
[('20', 1), ('Macchiato', 2), ('กาแฟ', 1), ('กาแฟร้อน', 1), ('กิน', 1), ('คน', 1), ('ครึ่ง', 1), ('ความคิด', 1), ('ชอบ', 1), ('ดื่ม', 1), ('ตอน', 1), ('ทาน', 1), ('นึง', 1), ('บาท', 1), ('ปริมาณ', 1), ('มีความสุข', 1), ('ลา', 1), ('สั่ง', 1), ('หนัก', 1), ('หลังจากนี้', 1), ('เข้าไป', 1), ('เจอ', 1), ('เดิน', 1), ('เดิม', 1), ('เป็นประจำ', 1), ('เฟล', 2), ('แก้ว', 2), ('โบราณ', 1)]


In [15]:
sum_review_word_freq = {}
for review_word_freq in word_frequencies:
    for word_freq in review_word_freq:
        if word_freq[0] in sum_review_word_freq:
            sum_review_word_freq[word_freq[0]] += word_freq[1]
        else:
            sum_review_word_freq[word_freq[0]] = word_freq[1]


range | word | sum. freq 
:-----|:----:|----:
1 | อร่อย 513 รสชาติ 414 ดี 366 ชอบ 292 รส 144 | 1729
2 | กาแฟ 418 หอม 140 | 558
3 | ทาน 417 อาหาร 352 กิน 343 | 1112
4 | เมนู 311 สั่ง 303 จาน 198 เลือก 130 | 942
5 | ราคา 280 บาท 262 | 542
6 | ชา	265 | 265
7 | คน	233 นั่ง 207 บรรยากาศ 152 โต๊ะ 112 | 704
8 | หวาน 211 เค้ก 136 | 347
9 | ลอง	178 ดู	149 | 327
10 | นม	155 | 155
11 | เนื้อ 140 | 140
12 | น้ำ 123 เครื่องดื่ม 123| 246
13 | บริการ 119 สำหรับ 109 | 228
14 | หมู 110 | 110
15 | ไข่มุก 109 | 109


## **Topic Modeling**

In [16]:
print(sum_review_word_freq)
file=open(review_file_path_name + '-freq.csv', 'w', encoding='utf8')
file.write('word,freq\n')
for key in sum_review_word_freq.keys():
    file.write(key + ',' + str(sum_review_word_freq[key]) + '\n')
file.close()

เปอร์เซ็นต์': 1, 'เพื่อนฝูง': 1, 'เรียงตัว': 1, 'แซม': 1, 'โซเชียล': 1, 'โมเดล': 1, 'ในที': 1, 'ไข่เจียว': 3, 'ไวไฟ': 1, '^_____^~': 1, '____': 1, 'กล': 1, 'ก้าว': 1, 'คุณชาย': 2, 'จ่ะ': 1, 'ญวน': 2, 'ปีใหม่': 3, 'พล่า': 1, 'ร่ำร้อง': 1, 'ลวกปาก': 1, 'ล่ม': 1, 'หรอ': 1, 'อืมมม': 1, 'อ้อย': 1, 'เด็ดดวง': 1, 'เนือง': 4, 'เเน่': 1, 'เเว๊น': 1, 'แหนม': 5, 'แอ๊พ': 1, 'โห๊ะ': 1, 'ได้โอกาส': 2, 'place': 1, 'pure': 1, 'ทั่ง': 1, 'น้ำหนัก': 2, 'ราชพฤกษ์': 1, 'สุขภาพ': 7, 'ออร์แกนิค': 1, 'โครงการ': 1, 'waffles': 1, 'ถา': 2, 'นค': 1, 'สวีท': 3, 'สิงห์': 1, 'ฮี้': 1, 'แผน': 1, 'โรแมนติก': 5, 'ต่อย': 2, 'ปชั่น': 1, 'ผึ้ง': 3, 'เสียว': 1, 'แหล่ง': 3, 'ขยาย': 2, 'ติดกัน': 4, 'ราชบุรี': 1, 'เป็นครั้งแรก': 3, 'เผอิญ': 2, 'เวอร์': 4, 'แซบ': 3, 'ติดปาก': 2, 'บริหาร': 1, 'วันอาทิตย์': 4, 'วิศวกร': 1, 'สถาปนิก': 1, 'สะเดา': 2, 'สิงห์บุรี': 1, 'สี่แยก': 2, 'อิ่มแปร้': 1, 'แกงเลียง': 1, 'แมคโคร': 1, 'แมงดา': 3, 'แ่ซ่บ': 1, 'โฮ': 1, '115': 1, 'กระเป๋า': 2, 'กระเป๋าสตางค์': 1, 'ดีแทค': 1, 'นึกดู': 1, 'ปอง': 2,

PermissionError: [Errno 13] Permission denied: 'Wongnai Reviews - Small-freq.csv'

In [17]:
num_topics = list(range(20)[1:])
num_keywords = 15
chunksize = 4000 # size of the doc looked at every pass
passes = 20 # number of passes through documents
iterations = 50
eval_every = 1  # Don't evaluate model perplexity, takes too much time.

# Make a index to word dictionary.
temp = dictionary[0]  # This is only to "load" the dictionary.
id2word = dictionary.id2token

%time model = gensim.models.LdaModel(corpus=gensim_corpus, id2word=id2word, chunksize=chunksize, \
                       alpha='auto', eta='auto', \
                       iterations=iterations, num_topics=num_topics[-1], \
                       passes=passes, eval_every=eval_every)

Wall time: 11.7 s


In [18]:
pyLDAvis.gensim_models.prepare(model, gensim_corpus, dictionary)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
13    -0.098607 -0.039982       1        1  15.422746
7     -0.114350  0.004649       2        1  15.123174
11    -0.093584 -0.113511       3        1  12.403406
5     -0.112761  0.102783       4        1   9.981637
3     -0.115780  0.044157       5        1   9.906455
16    -0.033633 -0.061143       6        1   5.913142
9     -0.040083 -0.048151       7        1   5.754216
2     -0.059885  0.027579       8        1   4.694215
17    -0.016265  0.072219       9        1   4.074912
15    -0.026557 -0.061219      10        1   3.292118
4     -0.020222  0.020141      11        1   2.792012
18     0.049376 -0.004633      12        1   1.711557
10     0.057369  0.101094      13        1   1.453501
1      0.165930 -0.041944      14        1   1.347040
6      0.079902 -0.020103      15        1   1.339864
8      0.071690 -0.018890      16        1   1.333794
12     0.095229  0.001124      17        1   1.172959
14     0.116142  0.054327      18        1   1.165068
0      0.096089 -0.018496      19        1   1.118184, topic_info=       Term        Freq       Total Category  logprob  loglift
2      กาแฟ  353.000000  353.000000  Default  30.0000  30.0000
153   อร่อย  451.000000  451.000000  Default  29.0000  29.0000
34       ชา  234.000000  234.000000  Default  28.0000  28.0000
36       ดี  321.000000  321.000000  Default  27.0000  27.0000
11      ทาน  365.000000  365.000000  Default  26.0000  26.0000
..      ...         ...         ...      ...      ...      ...
44     ราคา    1.146022  249.697930  Topic19  -5.7680  -0.8905
54     เมนู    1.145584  277.873975  Topic19  -5.7684  -0.9978
546    เค้ก    1.145565  115.595564  Topic19  -5.7684  -0.1207
4       กิน    1.145563  306.754445  Topic19  -5.7684  -1.0967
425  บริการ    1.145556  103.888540  Topic19  -5.7684  -0.0140

[1398 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
943       8  0.823487    -cake
1192      4  0.689947       00
1192      9  0.229982       00
625       1  0.146465       10
625       2  0.170876       10
...     ...       ...      ...
867      17  0.127020  ไอศครีม
4331     12  0.708051   ไออุ่น
5485     14  0.512876   ์กช้อค
3758     17  0.754814    ์เบอร
4434     15  0.510006       ••

[3367 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[14, 8, 12, 6, 4, 17, 10, 3, 18, 16, 5, 19, 11, 2, 7, 9, 13, 15, 1])

In [19]:
LDA_models = {}
LDA_topics = {}
for i in num_topics:
    %time LDA_models[i] = gensim.models.LdaModel(corpus=gensim_corpus, id2word=id2word, chunksize=chunksize, \
                       alpha='auto', eta='auto', \
                       iterations=iterations, num_topics=i, \
                       passes=passes, eval_every=eval_every)

    shown_topics = LDA_models[i].show_topics(num_topics=i, num_words=num_keywords, formatted=False)
    LDA_topics[i] = [[word[0] for word in topic[1]] for topic in shown_topics]

Wall time: 5.54 s
Wall time: 13 s
Wall time: 12.9 s
Wall time: 14.8 s
Wall time: 14.9 s
Wall time: 17.4 s
Wall time: 33.2 s
Wall time: 13.7 s
Wall time: 20.7 s
Wall time: 11.1 s
Wall time: 13.5 s
Wall time: 13.8 s
Wall time: 18.7 s
Wall time: 14.2 s
Wall time: 13.8 s
Wall time: 11.9 s
Wall time: 14.3 s
Wall time: 15.7 s
Wall time: 13.2 s


In [20]:
for topics in LDA_topics:
    print(str(len(LDA_topics[topics])) + ': ' + str(LDA_topics[topics]))

'ดี', 'บาท', 'นั่ง', 'ชอบ', 'ราคา', 'ไข่มุก', 'หวาน', 'ลอง'], ['จาน', 'ทาน', 'อาหาร', 'กิน', 'ปลา', 'สั่ง', 'ดี', 'ชิ้น', 'รส', 'อร่อย', 'บาท', 'โต๊ะ', 'เนื้อ', 'ญี่ปุ่น', 'เมนู'], ['กิน', 'อร่อย', 'อาหาร', 'เมนู', 'คน', 'รสชาติ', 'กาแฟ', 'สั่ง', 'ดี', 'ทาน', 'ชอบ', 'จาน', 'นะคะ', 'บาท', 'ซอส'], ['กาแฟ', 'ดี', 'อาหาร', 'รสชาติ', 'ทาน', 'เมนู', 'กก', 'สั่ง', 'ชอบ', 'เค้ก', 'คน', 'อร่อย', 'บริการ', 'บรรยากาศ', 'ชา']]
8: [['รสชาติ', 'อาหาร', 'ดี', 'เมนู', 'ทาน', 'กิน', 'กาแฟ', 'เล่น', 'สั่ง', 'จีน', 'แมว', 'คน', 'นั่ง', 'เนื้อ', 'อร่อย'], ['อร่อย', 'ทาน', 'กาแฟ', 'รสชาติ', 'เมนู', 'ดี', 'อาหาร', 'กิน', 'ชอบ', 'คน', 'สั่ง', 'จาน', 'เค้า', 'ลอง', 'หวาน'], ['คน', 'กิน', 'รสชาติ', 'ชอบ', 'ดี', 'สั่ง', 'ทาน', 'ซื้อ', 'กาแฟ', 'อาหาร', 'เค้า', 'อร่อย', 'นั่ง', 'บริการ', 'หวาน'], ['กาแฟ', 'อร่อย', 'เค้ก', 'ราคา', 'ดี', 'สั่ง', 'บาท', 'ทาน', 'นั่ง', 'อาหาร', 'เมนู', 'รสชาติ', 'ชา', 'บรรยากาศ', 'ลอง'], ['อาหาร', 'กาแฟ', 'รสชาติ', 'ดี', 'ทาน', 'สั่ง', 'อร่อย', 'ราคา', 'กิน', 'เมนู', 'ชอบ', 'นั่ง', '

In [21]:
def jaccard_similarity(topic_1, topic_2):
    """
    Derives the Jaccard similarity of two topics

    Jaccard similarity:
    - A statistic used for comparing the similarity and diversity of sample sets
    - J(A,B) = (A ∩ B)/(A ∪ B)
    - Goal is low Jaccard scores for coverage of the diverse elements
    """
    intersection = set(topic_1).intersection(set(topic_2))
    union = set(topic_1).union(set(topic_2))
                    
    return float(len(intersection))/float(len(union))

In [22]:
import numpy as np
LDA_stability = {}
for i in range(0, len(num_topics)-1):
    jaccard_sims = []
    for t1, topic1 in enumerate(LDA_topics[num_topics[i]]): # pylint: disable=unused-variable
        sims = []
        for t2, topic2 in enumerate(LDA_topics[num_topics[i+1]]): # pylint: disable=unused-variable
            sims.append(jaccard_similarity(topic1, topic2))    
        
        jaccard_sims.append(sims)    
    
    LDA_stability[num_topics[i]] = jaccard_sims
                
mean_stabilities = [np.array(LDA_stability[i]).mean() for i in num_topics[:-1]]

In [23]:
coherences = [gensim.models.coherencemodel.CoherenceModel(model=LDA_models[i], texts=texts, dictionary=dictionary, coherence='c_v').get_coherence() for i in num_topics]

In [24]:
print(coherences)

[0.35758584250879044, 0.34565225912175956, 0.33865086902760905, 0.33785541669186514, 0.32555708766436275, 0.329749960756222, 0.30953984723255507, 0.3373854891310272, 0.3225519968209537, 0.32751048132187804, 0.3297209697691217, 0.33127073646301053, 0.3006992870279559, 0.3288627181459295, 0.35441859010696597, 0.32487009310240356, 0.3165379036878382, 0.28659349564659675, 0.3131180688718768]


In [25]:
coh_sta_diffs = [coherences[i] - mean_stabilities[i] for i in range(num_keywords)[:-1]] # limit topic numbers to the number of keywords
coh_sta_max = max(coh_sta_diffs)
coh_sta_max_idxs = [i for i, j in enumerate(coh_sta_diffs) if j == coh_sta_max]
ideal_topic_num_index = coh_sta_max_idxs[0] # choose less topics in case there's more than one max
ideal_topic_num = num_topics[ideal_topic_num_index]
print('The ideal number of topics roughly through the difference between the coherence and stability per number of topics: ' + str(ideal_topic_num))

The ideal number of topics roughly through the difference between the coherence and stability per number of topics: 14


In [26]:
import matplotib.pyplot as plt

plt.figure(figsize=(20,10))
ply.plot(x=num_topics[:-1], y=mean_stabilities, label='Average Topic Overlap')
ply.plot(x=num_topics[:-1], y=coherences, label='Topic Coherence')

ply.plot(x=ideal_topic_num, label='Ideal Number of Topics', color='black')
plt.legend(fontsize=20)
plt.show() 

ModuleNotFoundError: No module named 'matplotib'

## Create understanable topic ##

In [27]:
# get from most 10 tokens
def convert_to_topic(x, num_keywords):
  topic = ""
  for i in range(len(x)):
    topic += x[i][0]
    if i >= num_keywords - 1:
      break;
  return topic

In [28]:
# gen. topic name
gen_topic_names = []
sel_model = LDA_models[ideal_topic_num]
for i in range(sel_model.num_topics):
  topic_name = convert_to_topic(sel_model.show_topic(i), num_keywords)
  gen_topic_names.append(topic_name)

print (gen_topic_names)

['กินอร่อยราคาบาทนะคะดีเมนูเค้กชอบคน', 'เป็ดโต๊ะกาแฟบาทฟี่ริร้อยคอโอเจลลี่', 'ทานอร่อยกาแฟสั่งดีรสชาติกินรสคนชอบ', 'จานรสชาติอร่อยอาหารทานเมนูสั่งดีบาทปลา', 'กุ้งชาดแป้งกาแฟน่ารักแมวทานเกร็ดพิมพ์นุ่ม', 'ทานอร่อยรสชาติสาขาบาทเมนูสั่งอาหารดีกิน', 'อาหารรสชาติกาแฟกินดีอร่อยน้ำสั่งปั่นราคา', 'ชาไข่มุกนมรสชาติหวานแก้วบาททานเลือกเครื่องดื่ม', 'กาแฟอร่อยอาหารชอบดีรสชาติเมนูทานราคาบาท', 'ทานอาหารเมนูดีอร่อยกาแฟรสชาติราคาชอบรังนก', 'เค้าอาหารคนกินรสชาติระดับดีบริการเผ็ดที่นี่', 'กาแฟรสชาติกกทานพญาไทอร่อยนรสิงห์พระราชวังสะพานนุ่ม', 'กาแฟดีเมนูนั่งรสชาติลองสั่งกินราคาชอบ', 'อร่อยทอดดีผักรสชาติอาหารกุ้งเมนูทานขนมปัง']


## Categorize reviews

In [29]:
df['topics'] = df['Review_tokenized'].apply(lambda x: sel_model.get_document_topics(dictionary.doc2bow(x.split(',')))[0][0])
df['score'] = df['Review_tokenized'].apply(lambda x: sel_model.get_document_topics(dictionary.doc2bow(x.split(',')))[0][1])
df['topic_token'] = df['Review_tokenized'].apply(lambda x: sel_model.show_topic(sel_model.get_document_topics(dictionary.doc2bow(x.split(',')))[0][0]))
df['topic_name'] = df['topic_token'].apply(lambda x:convert_to_topic(x, num_keywords))

In [30]:
pd.set_option('display.max_columns', None)
df['Review_short'] =  df['Review'].apply(lambda x: x[0:200])
df[['Review_short', 'topics', 'topic_name','score']].tail(30)

,Review_short,topics,topic_name,score
270,ร้านยำแซ่บอยู่ชั้น6 ห้างเซ็นทรัลพระราม9\r\nร้า...,2,ทานอร่อยกาแฟสั่งดีรสชาติกินรสคนชอบ,0.512569
271,หลายคนคงเคยได้ยินข่าวว่าทานชานมไข่มุกมากๆไม่ดี...,2,ทานอร่อยกาแฟสั่งดีรสชาติกินรสคนชอบ,0.834337
272,เรือนกาแฟ By Pupe' ร้านกาแฟชิลๆ ของดีเจปูเป้ ว...,0,กินอร่อยราคาบาทนะคะดีเมนูเค้กชอบคน,0.782705
273,ครั้งแรก ก็ love เลย ไม่บอกอย่างนี้คงไม่ได้ ร้...,2,ทานอร่อยกาแฟสั่งดีรสชาติกินรสคนชอบ,0.870719
274,เห็นหลายคนบอกว่าอร่อยอ้ะนะ เลยมาชิมมาชมมาช้อปไ...,2,ทานอร่อยกาแฟสั่งดีรสชาติกินรสคนชอบ,0.861988
275,เห็นรีวิว เลยมาลองทาน ร้านอยู่ไม่ไกลจากบ้าน\r\...,2,ทานอร่อยกาแฟสั่งดีรสชาติกินรสคนชอบ,0.182066
276,ร้านตำแหล เป็นร้านส้มตำของเหล่าดาราในดวงใจ คุณ...,3,จานรสชาติอร่อยอาหารทานเมนูสั่งดีบาทปลา,0.993842
277,ร้่านฝ้ายเป็นร้านคู่กับเมืองเชียงใหม่มานาน อยู...,8,กาแฟอร่อยอาหารชอบดีรสชาติเมนูทานราคาบาท,0.991582
278,ร้านตำนานป่าเป็นร้านอาหารชื่อดังของจังหวัดระยอ...,3,จานรสชาติอร่อยอาหารทานเมนูสั่งดีบาทปลา,0.131168
279,รีวิวในครั้งนี้ไม่ใช่สเต็กในร้าน steak lao นะค...,2,ทานอร่อยกาแฟสั่งดีรสชาติกินรสคนชอบ,0.994122


In [31]:
df[['Review_short', 'topics', 'topic_name','score']].to_csv(review_file_path_name + '-topics.csv', encoding='utf8')
